# 第五章 边缘检测  
## 5.2 边缘检测的数学模型  
- 给定一副图像$I$，边缘检测的目标是计算出图像中每一点$(i,j)$处的边缘强度$E_{s}(i,j|I)$和边缘方向$E_{θ}(i,j|I)$  
- 基于图像的离散性，图像像素的一阶微分形式可以用差分形式来表示为：  
    $$
    I_{x}(x,y)=\frac{I(i+1,j)-I(i-1,j)}{2}
    $$  
    $$
    I_{y}(x,y)=\frac{I(i,j+1)-I(i,j-1)}{2}
    $$  
    其中，$I_{x}(x,y)$和$I_{y}(x,y)$分别是图像像素值关于位置的二元函数在像素$(i，j)$处沿$x$轴方向和沿$y$轴方向的一阶偏导数，也称之为各自轴方向上的梯度幅值。上式可变形为：
    $$
    I_{x}(x,y)=\frac{1}{2}\sum_{m=-1}^{1}\sum_{n=-1}^{1}I(i-m,j-n)K_{x}(m,n)
    $$  
    $$
    I_{y}(x,y)=\frac{1}{2}\sum_{m=-1}^{1}\sum_{n=-1}^{1}I(i-m,j-n)K_{x}(m,n)
    $$  
    令矩阵$K_{x}=\begin{bmatrix}0&0&0\\+1&0&-1\\0&0&0\end{bmatrix}$，$K_{y}=\begin{bmatrix}0&+1&0\\0&0&0\\0&-1&0\end{bmatrix}$，令矩阵$I_{x}=(I_{x}(i,j))$，$I_{y}=(I_{y}(i,j))$，则上式可写为
    $$
    I_{x}=I*K_{x}, I_{y}=I*K_{y}
    $$  
    即可以通过卷积计算图像的一阶差分，两个卷积核也成为差分算子  
- 边缘强度和边缘方向可分别计算为  
    $$\mathbf{E}_s[i,j|\mathbf{I}]=\|\nabla\mathbf{I}[i,j]\|=\sqrt{\mathbf{I}_x^2[i,j]+\mathbf{I}_y^2[i,j]}$$
    $$\mathbf{E}_{\theta}[i,j|\mathbf{I}]=\tan^{-1}(\frac{\mathbf{I}_y[i,j]}{\mathbf{I}_x[i,j]})+\frac{\pi}{2}$$
## 5.3 边缘检测算法  
- 边缘检测算法通常有3个步骤：图像平滑、图像梯度计算、图像边缘定位，由于前两步都可通过卷积完成，所以可以结合为一个步骤，同时进行平滑和梯度计算  

### 5.3.1 Sobel边缘检测算法  
- Sobel算子  
    $$\mathbf{K}_x = \begin{bmatrix}
   +1 & 0 & -1 \\
   +2 & 0 & -2 \\
   +1 & 0 & -1
  \end{bmatrix},\quad
  \mathbf{K}_y = \begin{bmatrix}
   +1 & +2 & +1 \\
   0 & 0 & 0 \\
   -1 & -2 & -1
  \end{bmatrix}
    $$
    其中，选取沿$X$轴方向的Sobel算子，将其拆分为两个向量相乘：  
    $$
    \mathbf{K}_x = \begin{bmatrix}
    +1 & 0 & -1 \\
    +2 & 0 & -2 \\
    +1 & 0 & -1
    \end{bmatrix} = \begin{bmatrix} 
    1 \\ 2 \\ 1
    \end{bmatrix}\begin{bmatrix} 
    +1 & 0 & -1
    \end{bmatrix}
    $$  
    前者类似于一个高斯滤波器(中间大两边小)，目的是对图像中的噪声进行抑制；后者是一个差分算子，作用于图像得到沿$x$轴的梯度幅值，也就是垂直方向上的边缘强度  

### 5.3.2 Canny边缘检测算法  
- Canny边缘检测算法步骤：(1)图像平滑 (2)图像梯度计算 (3)边缘强度非极大值抑制 (4)基于滞后的阈值化的边缘定位  
- 边缘强度非极大值抑制的作用：直接从梯度得到的边缘线比较粗，且模糊，利用非极大值抑制将边缘细化  
- 基于滞后的阈值化处理的边缘定位：使用双阈值，防止边缘断裂  
- **Canny算子是应用最广泛的一种边缘检测算子，通过改变卷积核大小调节对图像的平滑程度，输出多尺度的边缘检测结果，并通过非极大值抑制和滞后的阈值化方法分别提升边缘检测的位置准确度性和对噪声的鲁棒性。**  

In [ ]:
"""""
@Author     :   jiguotong
@Contact    :   1776220977@qq.com
@site       :   
-----------------------------------------------
@Time       :   2025/5/14
@Description:   第5章 对图像应用Sobel算子边缘检测算法
"""""
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import *

# 输入图像
img = cv2.imread('lena.jpeg')

# x方向Sobel算子
kx = np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]
])

# y方向Sobel算子
ky = np.array([
    [1, 2, 1],
    [0, 0, 0],
    [-1,-2,-1]
])

# 展示输入图像
plt.imshow(img[:, :, ::-1])
plt.axis('off')

# x方向Sobel算子与图像进行卷积
conv_x = conv_2d(img, kx)
x_gradient = conv_x.conv()
conv_x.plot()

# y方向Sobel算子与图像进行卷积
conv_y = conv_2d(img, ky)
y_gradient = conv_y.conv()
conv_y.plot()

# 基于以上梯度计算边缘强度, 使用不开平方的近似值相加得到边缘强度
E = abs(x_gradient) + abs(y_gradient)
plt.imshow(E[:, :, ::-1])
plt.axis('off')

In [ ]:
"""""
@Author     :   jiguotong
@Contact    :   1776220977@qq.com
@site       :   
-----------------------------------------------
@Time       :   2025/5/14
@Description:   第5章 对图像应用Canny算子边缘检测算法，逐步骤
"""""
from utils import *
from PIL import Image
import cv2
# 设定高斯核的大小与方差
kernel_size = 5
sigma = 1.5

# 导入一张图像
img = Image.open('lena.jpeg')

# 将其转为灰度图像
img = img.convert('L')
img = np.array(img)

# 构造高斯卷积核
kernel = gaussian_kernel(kernel_size, sigma)

# 与图像进行卷积
conv_= conv_2d(img, kernel)
smoothed = conv_.conv()

# 图像边缘强度与角度
E, theta = gradient(smoothed)
plot_image(np.uint8(E), 'Gradient magnitude')

nms = non_maximum_suppression(E, theta)
plot_image(np.uint8(nms), 'Non-maximum suppressed')

low_threshold = 3
high_threshold = 6

strong_edges, weak_edges = double_thresholding(nms, 
                    high_threshold, low_threshold)

# 返回强弱边缘叠加的边缘图
edges = strong_edges * 1.0 + weak_edges * 0.5

plot_image(strong_edges, 'Strong Edges')
plot_image(edges, 'Strong+Weak Edges')

# 获得最终边缘图
edges = link_edges(strong_edges, weak_edges)
plot_image(edges, 'Final image')

In [ ]:
"""""
@Author     :   jiguotong
@Contact    :   1776220977@qq.com
@site       :   
-----------------------------------------------
@Time       :   2025/5/14
@Description:   第5章 对图像应用Canny算子边缘检测算法，直接调用封装的接口
"""""
from utils import *
from PIL import Image
# 导入一张图像
img = Image.open('lena.jpeg')

# 将其转为灰度图像
img = img.convert('L')
img = np.array(img)
plot_image(img, 'Original image')
canny_img = canny(img)
plot_image(canny_img, 'Edges of image')

canny2 = canny(img, kernel_size=7, sigma=2)
plot_image(canny2, 'Kernel Size: 7, Sigma: 2')

canny2 = canny(img, kernel_size=9, sigma=2.5)
plot_image(canny2, 'Kernel Size: 9, Sigma: 2.5')

canny2 = canny(img, kernel_size=9, sigma=3)
plot_image(canny2, 'Kernel Size: 9, Sigma: 3')